<a href="https://www.kaggle.com/code/prasadposture121/text-generation-project?scriptVersionId=101858496" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Import Dependencies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data
# loading data and opening our input data in the form of a txt file
# Project Gutenburg/berg is where the data can be found
file=open("../input/frankenstein/frankenstein.txt").read()

In [3]:
# tokenization
# standardization
#what is tokenization? Tokenization is the process of breaking a stream of text up into words phrases symbols or some meaningful elements
def tokenize_words(input):
    input=input.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(input)
    filtered=filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)
processed_inputs=tokenize_words(file)

In [4]:
#chara to numbers
chars=sorted(list(set(processed_inputs)))
char_to_num=dict((c,i) for i,c in enumerate(chars))

In [5]:
#check if the words to char or chars to num has worked
input_len=len(processed_inputs)
vocab_len=len(chars)
print("Total number of characters:", input_len)
print("Total vocab:",vocab_len)

Total number of characters: 269566
Total vocab: 38


In [6]:
#sege length
seq_length=100
x_data=[]
y_data=[]

In [7]:
#loop through the sequence
for i in range(0,input_len-seq_length,1):
    in_seq=processed_inputs[i:i+seq_length]
    out_seq=processed_inputs[i+seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns=len(x_data)
print("Total Patterns:", n_patterns)

Total Patterns: 269466


In [8]:
# convert input sequence to np array and so on
X=numpy.reshape(x_data,(n_patterns, seq_length,1))
X=X/float(vocab_len)

In [9]:
# One-hot encoding
y=np_utils.to_categorical(y_data)

In [10]:
#Creating the model
model=Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))

2022-07-27 07:08:06.123182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 07:08:06.260013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 07:08:06.260785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 07:08:06.262114: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [11]:
#Compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [12]:
#saving the weights
filepath='model_weights_saved.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='loss',verbose=1,save_best_only=True,mode='min')
desired_callbacks=[checkpoint]

In [13]:
#fit the model and let it train
model.fit(X,y,epochs=10,batch_size=256,callbacks=desired_callbacks)

2022-07-27 07:08:09.974119: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-07-27 07:08:14.057128: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1053/1053 [==============================] - 71s 62ms/step - loss: 2.7652

Epoch 00001: loss improved from inf to 2.76518, saving model to model_weights_saved.hdf5
Epoch 2/10
1053/1053 [==============================] - 66s 62ms/step - loss: 2.4276

Epoch 00002: loss improved from 2.76518 to 2.42755, saving model to model_weights_saved.hdf5
Epoch 3/10
1053/1053 [==============================] - 66s 63ms/step - loss: 2.1707

Epoch 00003: loss improved from 2.42755 to 2.17065, saving model to model_weights_saved.hdf5
Epoch 4/10
1053/1053 [==============================] - 66s 63ms/step - loss: 1.9995

Epoch 00004: loss improved from 2.17065 to 1.99952, saving model to model_weights_saved.hdf5
Epoch 5/10
1053/1053 [==============================] - 66s 63ms/step - loss: 1.8855

Epoch 00005: loss improved from 1.99952 to 1.88548, saving model to model_weights_saved.hdf5
Epoch 6/10
1053/1053 [==============================] - 66s 63ms/step - loss: 1.8087

Epoch 00006: loss improved from 1.

In [14]:
#recompile the model with saved weights
filename='model_weights_saved.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [15]:
# output of the models back into characters
num_to_char=dict((i,c) for i,c in enumerate(chars))

In [16]:
#random seed to help generate
start=numpy.random.randint(0,len(x_data)-1)
pattern=x_data[start]
print("Random Seed :")
print("\"",''.join([num_to_char[value] for value in pattern]),"\"")

Random Seed :
" idual project gutenberg tm electronic work derived public domain contain notice indicating posted pe "


In [17]:
# generate the text
for i in range(1000):
    x=numpy.reshape(pattern,(1,len(pattern),1))
    x=x/float(vocab_len)
    prediction=model.predict(x, verbose=0)
    index=numpy.argmax(prediction)
    result=num_to_char[index]
    seg_in=[num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern=pattern[1:len(pattern)]

rceived see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see see 

The End